In [22]:
import wandb
import json
import pandas as pd

project = "consistency"
entity = "itl"

api = wandb.Api()
runs = api.runs(f"{entity}/{project}")

dfs = []
# download all tables
for run in runs:
    artifacts = run.logged_artifacts()
    for artifact in artifacts:
        print(artifact.name)
        table_dir = artifact.download()
        table_path = f"{table_dir}/evaluated_statements.table.json"
        with open(table_path) as file:
            json_dict = json.load(file)
        df = pd.DataFrame(json_dict["data"], columns=json_dict["columns"])
        df["model"] = run.name
        dfs.append(df)

df = pd.concat(dfs)

run-mazvi7la-evaluated_statements:v0


wandb:   1 of 1 files downloaded.  


In [27]:
def parse_validator(x):
    if "both" in x.lower():
        return "both"
    elif "one" in x.lower():
        return "one"
    else:
        return "unknown"

df["eval_text_val"] = df["eval_text"].apply(parse_validator) == "both"
df["eval_image_val"] = df["eval_image"].apply(parse_validator) == "both"
df["eval_both_val"] = df["eval_both"].apply(parse_validator) == "both"

In [29]:
df.groupby("generated_with").agg(
    {"eval_text_val": "mean", "eval_image_val": "mean", "eval_both_val": "mean"}
)

,eval_text_val,eval_image_val,eval_both_val
generated_with,,,
both,0.810078,0.779070,0.986434
image,0.757170,0.873805,0.961759
text,0.901804,0.783567,0.995992
